In [55]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import re
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

In [56]:
# TRAIN_DF = pd.read_csv("Data/TRAIN_DF.csv", index_col=0)
TEST_DF = pd.read_csv("C:\\MachineLearning\\UniTrier\\RCS\\twon-hate-speech-detection\\Data\\knowledge_base_data\\TEST_DF.csv", index_col=1)

In [57]:
TEST_DF.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2016 entries, 10071 to 6853
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0.1    2016 non-null   int64  
 1   Input           2016 non-null   object 
 2   Response        2016 non-null   object 
 3   Time Taken (s)  2016 non-null   float64
 4   tweet_text      2016 non-null   object 
 5   key_features    2016 non-null   object 
 6   label           2016 non-null   object 
 7   target          1819 non-null   object 
 8   Unnamed: 0      1115 non-null   float64
dtypes: float64(2), int64(1), object(6)
memory usage: 157.5+ KB


In [58]:
TEST_DF['label'].value_counts()

label
normal        672
hatespeech    672
offensive     672
Name: count, dtype: int64

In [59]:
#TEST_DF.set_index('Row Number', inplace=True)

In [60]:
# this dataset have predictions of all LLMs combined (WITHOUT RAG)
# change the absolute path or use os chdir twice to go back and give path to llms\combined_all_llm_classification_predictions_no_RAG.csv
non_rag_combined_results_df = pd.read_csv("C:\\MachineLearning\\UniTrier\\RCS\\twon-hate-speech-detection\\classification_experiments\\llms\\combined_all_llm_classification_predictions_no_RAG.csv", index_col=0)

In [61]:
# these datasets have predictions (WITH RAG)
hermes_rag_df = pd.read_csv("classified_test_df_Hermes-3-Llama-3.1-70B-Q5_K_S.csv", index_col=1)
mistral_rag_df = pd.read_csv("classified_test_df_mistral-7b-instruct-v0.2-q8_0.csv", index_col=1)
llama_rag_df = pd.read_csv("classified_test_df_Llama-3.3-70B-Instruct-Q5-K.csv", index_col=1)

In [22]:
df2 = hermes_rag_df
df1 = llama_rag_df

In [62]:
TEST_DF = TEST_DF.merge(
    hermes_rag_df[['RAG_Hermes-3-Llama-3.1-70B-Q5_K_S']], how='left', left_index=True, right_index=True
)

TEST_DF = TEST_DF.merge(
    llama_rag_df[['RAG_Llama-3.3-70B-Instruct-Q5-K']], how='left', left_index=True, right_index=True
)
TEST_DF = TEST_DF.merge(
    mistral_rag_df[['RAG_mistral:7b-instruct-v0.2-q8_0']], how='left', left_index=True, right_index=True
)

TEST_DF = TEST_DF.merge(
    non_rag_combined_results_df[['Hermes-3-Llama-3.1-70B-Q5_K_S', 'llama3.3:70B-Instruct-Q2_K', 'llama3.3:70b-instruct-q6_K-SEEN_DATA', 'llama3.3:70b-instruct-q6_K', 'mistral:7b-instruct-v0.2-q8_0']],
    how='left', left_index=True, right_index=True
)

In [69]:
TEST_DF.shape

(2016, 17)

In [63]:
TEST_DF.isna().sum()

Unnamed: 0.1                              0
Input                                     0
Response                                  0
Time Taken (s)                            0
tweet_text                                0
key_features                              0
label                                     0
target                                  197
Unnamed: 0                              901
RAG_Hermes-3-Llama-3.1-70B-Q5_K_S         0
RAG_Llama-3.3-70B-Instruct-Q5-K           0
RAG_mistral:7b-instruct-v0.2-q8_0         0
Hermes-3-Llama-3.1-70B-Q5_K_S             0
llama3.3:70B-Instruct-Q2_K                0
llama3.3:70b-instruct-q6_K-SEEN_DATA      0
llama3.3:70b-instruct-q6_K                0
mistral:7b-instruct-v0.2-q8_0             0
dtype: int64

In [64]:
# data cleaning
normalization_map = {
    "offensive": "offensive",
    "Offensive": "offensive",
    "[offensive]": "offensive",
    "[Offensive]": "offensive",
    "<assistant> offensitive": "offensive",
    "offensive.": "offensive",
    "<assistant> offensive": "offensive",
    
    "hate speech": "hatespeech",
    "Hatespeech": "hatespeech",
    "hate speech"
    "hatespeech": "hatespeech",
    "Hate Speech": "hatespeech",
    "Hate speech": "hatespeech",
    "Hate speech.": "hatespeech",
    "hate speech.": "hatespeech",
    "hatedspeech": "hatespeech",
    "hatemspeech": "hatespeech",
    "haterspeech": "hatespeech",
    "[hate speech]": "hatespeech",
    "[Hate Speech]": "hatespeech",
    "<hate speech>": "hatespeech",
    "'hatespeech'": "hatespeech",
    "hatredspeech": "hatespeech",
    "**Hate Speech**": "hatespeech",
    "'hate speech'": "hatespeech",
    "<hatespeech>": "hatespeech",
    "[hatespeech]": "hatespeech",
    
    "normal": "normal",
    "Normal": "normal",
    "Normal.": "normal",
    "[normal]": "normal",
    "[Normal]": "normal",
    
    # Handle cases where the label is inside a list-like structure
    "[Userinput, \"normal\", 1]": "normal",
    "[Userinput, \"offensive\", 1]": "offensive",
    "[Userinput, \"hatespeech\", 1]": "hatespeech",
}


columns_to_normalize = [
    "label",
    "Hermes-3-Llama-3.1-70B-Q5_K_S",
    "llama3.3:70B-Instruct-Q2_K",
    "llama3.3:70b-instruct-q6_K-SEEN_DATA",
    "llama3.3:70b-instruct-q6_K",
    "mistral:7b-instruct-v0.2-q8_0",

    #"RAG_llama3.3:70b-instruct-q6_K",
    #"RAG_mistral:7b-instruct-v0.2-q8_0",
    "RAG_Llama-3.3-70B-Instruct-Q5-K",
    "RAG_mistral:7b-instruct-v0.2-q8_0",
    "RAG_Hermes-3-Llama-3.1-70B-Q5_K_S",
]
def clean_text(text):
    if isinstance(text, str):  # Ensure it's a string before applying regex
        # return re.split(r'[.\n]', text, 1)[0].strip()  # Keep only the part before the first "." or "\n"
        return re.split(r'[.\n"]|\s*\(', text, 1)[0].strip()  # Split at ".", "\n", or " (" and keep the first part
    return text
    
for column in columns_to_normalize:
    TEST_DF[column] = TEST_DF[column].str.strip('"').str.strip().apply(clean_text)
    TEST_DF[column] = TEST_DF[column].replace(normalization_map)

In [65]:
TEST_DF.label.unique()

array(['normal', 'hatespeech', 'offensive'], dtype=object)

In [67]:
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

# List of classifier prediction columns
other_classifier_columns = [
    "Hermes-3-Llama-3.1-70B-Q5_K_S",
    "llama3.3:70B-Instruct-Q2_K",
    "llama3.3:70b-instruct-q6_K-SEEN_DATA",
    "llama3.3:70b-instruct-q6_K",
    "mistral:7b-instruct-v0.2-q8_0",
    "RAG_Llama-3.3-70B-Instruct-Q5-K",
    "RAG_Hermes-3-Llama-3.1-70B-Q5_K_S",
    "RAG_mistral:7b-instruct-v0.2-q8_0",
]

# True labels (ground truth)
y_test = TEST_DF["label"].astype(str).values  # Ensure string type

# Get valid labels from TEST_DF (ground truth)
valid_labels = set(TEST_DF["label"].dropna().unique())  # Ensure no NaN values

for column in other_classifier_columns:
    # Convert predictions to string and filter out NaN values
    y_pred_other = TEST_DF[column].astype(str).values
    y_test = TEST_DF["label"].astype(str).values  # Ground truth labels

    # Remove invalid predictions (i.e., predictions not in valid_labels)
    valid_indices = np.array([pred in valid_labels for pred in y_pred_other])
    y_test_valid = y_test[valid_indices]
    y_pred_other_valid = y_pred_other[valid_indices]

    # Compute accuracy only for valid predictions
    accuracy_other = accuracy_score(y_test_valid, y_pred_other_valid)
    print(f"\nAccuracy for {column}: {accuracy_other:.4f}")

    # Detailed classification report (ignore invalid predictions)
    print(f"\n{column} Classification Report:")
    print(classification_report(y_test_valid, y_pred_other_valid, labels=list(valid_labels), zero_division=0))


Accuracy for Hermes-3-Llama-3.1-70B-Q5_K_S: 0.5094

Hermes-3-Llama-3.1-70B-Q5_K_S Classification Report:
              precision    recall  f1-score   support

   offensive       0.43      0.27      0.33       670
      normal       0.70      0.37      0.48       672
  hatespeech       0.48      0.89      0.63       672

    accuracy                           0.51      2014
   macro avg       0.54      0.51      0.48      2014
weighted avg       0.54      0.51      0.48      2014


Accuracy for llama3.3:70B-Instruct-Q2_K: 0.4425

llama3.3:70B-Instruct-Q2_K Classification Report:
              precision    recall  f1-score   support

   offensive       0.37      0.16      0.22       672
      normal       0.83      0.21      0.34       672
  hatespeech       0.41      0.95      0.58       672

    accuracy                           0.44      2016
   macro avg       0.54      0.44      0.38      2016
weighted avg       0.54      0.44      0.38      2016


Accuracy for llama3.3:70b-instr